In [1]:
#Update your token
STUDENT_TOKEN = 'DANIEL KUMLIN'

# Server Code

In [20]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'

  def get_board(self):
    print(self.state)
    return json.loads(self.state)

  def actions(self):
    return []

  def print_game(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('\033[92mCreating new game...\033[0m')
    id = new_game(game_type, multi_player)

  print('\033[92mJoining game with id: %s\033[0m' % id)
  player = join_game(game_type, id)

  print('\033[92mPlaying as %s\033[0m' % player)

  game = game_state(game_type, id, GameClass)
  print('\033[91mWaiting for the other player to join...\033[0m')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('\033[94mdraw\033[0m')
      else:
        print('\033[92mYou won\033[0m' if game.player == player else '\033[91mYou lost\033[0m')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

# Stratego Game

In [21]:
from functools import reduce
from copy import copy, deepcopy
import json


class Stratego(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return self.state['possible_actions']

  def card_str(self, card):
    return '+'.join(map(str, card))

  def print_game(self):
    print('Board: ' + "\n".join("".join(row) for row in self.state['board']))
    if self.state['past_actions']:
      print('Last action: ' + str(self.state['past_actions'][-1]))
      print('Last two actions: ' + " ".join(str(action) for action in self.state["past_actions"][-2:]))

  def other_player(self):
    if self.player == 'X': return 'O'
    if self.player == 'O': return 'X'

  def get_board(self):
        # Since `state` is already a dictionary, just return it directly.
        return self.state

# Creating better representation of the board

## Making types for elements

In [10]:
from typing import List, Dict, Optional 
from dataclasses import dataclass
from enum import Enum

class PieceType(Enum):
    FLAG = 'F'
    BOMB = 'B'
    SCOUT = 's'
    SPY = 'S'
    GENERAL = 'G'
    MARSHAL = 'M'
    LAKE = 'L'
    UNKNOWN = 'O'
    EMPTY = '_'

class Player(Enum):
    X = 'X'
    O = 'O'
    NONE = '_'

@dataclass
class Position:
    row: int
    col: int
    
    def __post_init__(self) -> None:
        if not (0 <= self.row < 8 and 0 <= self.col < 8):
            raise ValueError(f"Invalid position: ({self.row}, {self.col})")

@dataclass
class Piece:
    type: PieceType
    value: int
    player: Optional[Player] = None
    revealed: bool = True
    
    def can_move(self) -> bool:
        return self.type not in [PieceType.FLAG, PieceType.BOMB, PieceType.LAKE]
    
    def can_attack(self, other: 'Piece') -> bool:
        if not other or other.type == PieceType.LAKE:
            return False
            
        if self.type == PieceType.SPY and other.type == PieceType.MARSHAL:
            return True
        if other.type == PieceType.BOMB:
            return self.type == PieceType.SCOUT
            
        return self.value >= other.value

## Defining virtual representation of board

In [25]:
from typing import Tuple, List

class StrategoBoard:
    def __init__(self, state: Optional[Dict] = None) -> None:
        self.board: List[List[Optional[Piece]]] = [
            [None for _ in range(8)] for _ in range(8)
        ]
        
        self.piece_values: Dict[PieceType, int] = {
            PieceType.FLAG: 0,
            PieceType.BOMB: 1,
            PieceType.SCOUT: 2,
            PieceType.SPY: 3,
            PieceType.GENERAL: 4,
            PieceType.MARSHAL: 5,
            PieceType.LAKE: -1,
            PieceType.UNKNOWN: None,
            PieceType.EMPTY: None
        }
        
        if state:
            self.load_from_state(state)


    def load_from_state(self, state: Dict) -> None:
        """Load board from game state"""
        for row in range(8):
            for col in range(8):
                piece_char = state['board'][row][col]
    
                if piece_char == '_':
                    # Empty space, no piece here
                    self.board[row][col] = None
                    continue
                
                if piece_char in ['X', 'O']:
                    # Player identifiers, not piece types, we skip these
                    continue
                
                # Normalize the piece character to uppercase to ensure it matches the enum values
                piece_char = piece_char.upper()
    
                try:
                    # Convert character to PieceType
                    piece_type = PieceType(piece_char)
    
                    # Determine player ownership
                    player = Player.X if state['player_board'][row][col] == 'X' else Player.O
    
                    # Add the piece to the board
                    self.board[row][col] = Piece(
                        type=piece_type,
                        value=self.piece_values[piece_type],
                        player=player
                    )
                except ValueError:
                    raise ValueError(f"Invalid piece character: {piece_char}")

    
    def make_move(self, move: Tuple[Position, Position]) -> Optional[Piece]:
        """
        Moves a piece from the start position to the end position.
        If there is an enemy piece at the destination, it is captured.
        """
        start_pos, end_pos = move
        moving_piece = self.get_piece(start_pos)

        if not moving_piece:
            raise ValueError(f"No piece at the starting position: {start_pos}")

        target_piece = self.get_piece(end_pos)

        # Determine if this is an attack
        if target_piece and target_piece.player != moving_piece.player:
            if moving_piece.can_attack(target_piece):
                # Attack is successful
                self.set_piece(end_pos, moving_piece)
                self.set_piece(start_pos, None)
                return target_piece  # Return the captured piece
            else:
                # Both pieces are destroyed if attack is unsuccessful
                self.set_piece(start_pos, None)
                self.set_piece(end_pos, None)
                return target_piece  # Return the destroyed enemy piece
        else:
            # No attack, just a move
            self.set_piece(end_pos, moving_piece)
            self.set_piece(start_pos, None)
            return None  # No piece was captured
    
    def undo_move(self, move: Tuple[Position, Position], captured_piece: Optional[Piece]) -> None:
        """
        Reverts a move made on the board.
        """
        start_pos, end_pos = move
        moving_piece = self.get_piece(end_pos)

        # Revert the piece back to its starting position
        self.set_piece(start_pos, moving_piece)

        # If a piece was captured, restore it
        if captured_piece:
            self.set_piece(end_pos, captured_piece)
        else:
            # Otherwise, just empty the target square
            self.set_piece(end_pos, None)

    def get_legal_moves(self, pos: Position) -> List[Position]:
        """
        This remains unchanged, but gives valid moves for a piece
        based on its type and the current board state.
        """
        piece = self.get_piece(pos)
        if not piece or not piece.can_move():
            return []

        moves: List[Position] = []
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Right, Left, Down, Up

        for dx, dy in directions:
            if piece.type == PieceType.SCOUT:
                # Scout can move multiple spaces
                current = Position(pos.row + dy, pos.col + dx)
                while self.is_valid_position(current) and not self.get_piece(current):
                    moves.append(current)
                    current = Position(current.row + dy, current.col + dx)
            else:
                # Other pieces move one space
                new_pos = Position(pos.row + dy, pos.col + dx)
                if self.is_valid_position(new_pos):
                    target = self.get_piece(new_pos)
                    if not target or piece.can_attack(target):
                        moves.append(new_pos)

        return moves

    def is_terminal(self) -> bool:
        """
        Checks if the game is over by seeing if either player’s flag is captured.
        """
        # Check if any player's flag is missing from the board
        has_flag_X = any(piece and piece.type == PieceType.FLAG and piece.player == Player.X for row in self.board for piece in row)
        has_flag_O = any(piece and piece.type == PieceType.FLAG and piece.player == Player.O for row in self.board for piece in row)

        return not (has_flag_X and has_flag_O)
    
    def __str__(self) -> str:
        """Pretty print the board"""
        return '\n'.join(
            ' '.join(str(piece.type.value) if piece else '_'
                    for piece in row)
            for row in self.board
        )


# Algorithms

In [15]:
class MinimaxStrategy:
    def __init__(self, max_depth: int = 4):
        self.max_depth = max_depth

    def choose_move(self, game: Stratego, board: StrategoBoard) -> List:
        """Entry point for move selection"""
        if game.state['phase'] == "place":
            return self._choose_placement(game.player)

        return self._minimax_decision(game, board)

    def _minimax_decision(self, game: Stratego, board: StrategoBoard) -> List:
        """Choose best move using minimax"""
        possible_moves = game.actions()
        best_move = None
        best_score = float('-inf')
        alpha = float('-inf')
        beta = float('inf')

        for move in possible_moves:
            # Apply move
            new_board = board.make_move(move)

            # Evaluate resulting state using minimax
            score = self._minimax(new_board, self.max_depth - 1, alpha, beta, False, game.player)

            if score > best_score:
                best_score = score
                best_move = move
            alpha = max(alpha, best_score)

        return best_move

    def _minimax(self, board: StrategoBoard, depth: int, alpha: float, beta: float, is_maximizing: bool, player: str) -> float:
        """Minimax algorithm with alpha-beta pruning"""
        if depth == 0 or board.is_terminal():
            return self._evaluate_position(board, player)

        if is_maximizing:
            max_eval = float('-inf')
            for move in board.get_legal_moves(player):
                new_board = board.make_move(move)
                eval = self._minimax(new_board, depth - 1, alpha, beta, False, player)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break  # Beta cutoff
            return max_eval
        else:
            min_eval = float('inf')
            opponent = 'O' if player == 'X' else 'X'
            for move in board.get_legal_moves(opponent):
                new_board = board.make_move(move)
                eval = self._minimax(new_board, depth - 1, alpha, beta, True, player)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break  # Alpha cutoff
            return min_eval

    def _evaluate_position(self, board: StrategoBoard, player: str) -> float:
        """Evaluate board position"""
        score = 0.0
        score += self._evaluate_material(board, player) * 10.0
        score += self._evaluate_position_control(board, player) * 5.0
        score += self._evaluate_flag_safety(board, player) * 50.0
        return score

    def _evaluate_material(self, board: StrategoBoard, player: str) -> float:
        """Count material advantage"""
        player_score = 0
        opponent_score = 0

        for row in board.board:
            for piece in row:
                if piece and piece.type != PieceType.LAKE:
                    if piece.player == player:
                        player_score += piece.value
                    else:
                        opponent_score += piece.value
        return player_score - opponent_score

    def _evaluate_position_control(self, board: StrategoBoard, player: str) -> float:
        """Evaluate control of important squares"""
        # Placeholder logic for important positions like enemy flag proximity, etc.
        control_score = 0
        # Implement custom logic for control score
        return control_score

    def _evaluate_flag_safety(self, board: StrategoBoard, player: str) -> float:
        """Evaluate flag protection"""
        flag_pos = None
        for row in range(8):
            for col in range(8):
                piece = board.get_piece(Position(row, col))
                if piece and piece.type == PieceType.FLAG and piece.player == player:
                    flag_pos = Position(row, col)
                    break

        if not flag_pos:
            return -float('inf')  # Flag is gone

        # Placeholder for flag safety score
        safety_score = 0
        # Add logic for checking how protected the flag is
        return safety_score


## Solver function

### Custom

In [18]:
def minimax_solver(game: Stratego) -> List:
    # Placement phase
    if game.state['phase'] == "place":
        if game.player == 'X':
            positions = [(y, x) for x in range(2, 6) for y in range(2)]
        else:
            positions = [(7 - y, x) for x in range(2, 6) for y in range(2)]
        
        pieces = ['F', 'G', 'M', 'S', 'S', 's', 'B', 'f']
        random.shuffle(pieces)
        
        final_pos = []
        for pos in positions:
            final_pos.append((pos[0], pos[1], pieces.pop()))
        
        print(final_pos)
        return final_pos
    
    # Moving phase
    else:
        current_board = StrategoBoard(state=game.get_board())
        minimax_strategy = MinimaxStrategy(max_depth=3)
        return minimax_strategy.choose_move(game, current_board)


### Random

In [6]:
def random_solver(game: Stratego):
  if game.state['phase'] == "place":
    if game.player == 'X':
      positions = [(y, x) for x in range(2, 6) for y in range(2)]
    else:
      positions = [(7 - y, x) for x in range(2, 6) for y in range(2)]
    pieces = ['F', 'G', 'M', 'S', 'S', 's', 'B', 'f']
    random.shuffle(pieces)
    final_pos = []
    for pos in positions:
      final_pos.append((pos[0], pos[1], pieces.pop()))
    print(final_pos)
    return final_pos
  else:
    return random.choice(game.actions())

# Start game loop

In [26]:
# Using the minimax_solver function in the game loop
game_loop(minimax_solver, Stratego, 'stratego', multi_player=False, id=None)

Creating new game...
Joining game with id: 17873
Playing as X
Waiting for the other player to join...
Board: ________
________
________
__L__L__
__L__L__
________
________
________
Making next move...
[(0, 2, 'f'), (1, 2, 'G'), (0, 3, 'B'), (1, 3, 'F'), (0, 4, 'S'), (1, 4, 'S'), (0, 5, 'M'), (1, 5, 's')]
b'Valid move'
Board: __fBSM__
__GFSs__
________
__L__L__
__L__L__
________
__OOOO__
__OOOO__
Making next move...


ValueError: Invalid piece character: f

In [7]:
game_loop(random_solver, Stratego, 'stratego', multi_player=False, id=None)

Creating new game...
b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'
b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'
Joining game with id: 17638
b'{"Message": "Error getting game, please retry"}'
Playing as X
Waiting for the other player to join...
Board: ________
________
________
__L__L__
__L__L__
________
________
________
Making next move...
<bound method Stratego.actions of <__main__.Stratego object at 0x113ca4da0>>
[(0, 2, 'M'), (1, 2, 's'), (0, 3, 'f'), (1, 3, 'B'), (0, 4, 'S'), (1, 4, 'G